In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.0/616.0 kB 9.9 MB/s eta 0:00:00


In [56]:
import torch
from torch import nn
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
import cv2
import optuna

In [57]:
# prompt: drive

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
# !unzip -q '/content/drive/MyDrive/learning machine learning/rice.zip'

In [59]:
class_wise_img_paths = glob.glob('/content/rice/*')

In [60]:
class_wise_img_paths

['/content/rice/Karacadag',
 '/content/rice/Basmati',
 '/content/rice/Arborio',
 '/content/rice/Jasmine',
 '/content/rice/Ipsala']

In [61]:
imgs_tensor = []
labels_tensor = []
curr_label = -1
for class_wise_path in class_wise_img_paths:
    curr_label += 1
    class_wise_image_paths = glob.glob(f'{class_wise_path}/*')
    for img_path in class_wise_image_paths[:1000]:
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))
        imgs_tensor.append(torch.from_numpy(img))
        labels_tensor.append(torch.tensor(curr_label))

In [62]:
imgs_tensor = torch.stack(imgs_tensor)

In [63]:
imgs_tensor = imgs_tensor.reshape(-1, 3, 224, 224)

In [64]:
imgs_tensor.shape

torch.Size([5000, 3, 224, 224])

In [65]:
labels_tensor = torch.stack(labels_tensor)

In [66]:
class PomodoroClassifierNetwork(nn.Module):

    def __init__(self, num_classes):
        super(PomodoroClassifierNetwork, self).__init__()
        self.conv_net = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, stride=2, padding=1),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=2, padding=1),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU()
        )
        self.fc = nn.Sequential(
            nn.Linear(in_features=3136, out_features=1024),
            nn.LeakyReLU(),
            nn.Linear(in_features=1024, out_features=512),
            nn.LeakyReLU(),
            nn.Linear(in_features=512, out_features=256),
            nn.LeakyReLU(),
            nn.Linear(in_features=256, out_features=128),
            nn.LeakyReLU(),
            nn.Linear(in_features=128, out_features=64),
            nn.LeakyReLU(),
            nn.Linear(in_features=64, out_features=num_classes),
            nn.Softmax(dim=-1),
        )

    def forward(self, x):
        batch_size = x.shape[0]
        # print(x.shape)
        x = self.conv_net(x)
        x = self.fc(x.view(batch_size, -1))
        # print(x.shape)
        return x

In [67]:
pcn = PomodoroClassifierNetwork(num_classes=5)

In [68]:
# experiment_name = "pomodoro_classification"
# if mlflow.get_experiment_by_name(experiment_name) == None:
#     mlflow.create_experiment(experiment_name)
# else:
#     mlflow.set_experiment(experiment_name)

In [71]:

# with mlflow.start_run() as run:

def objective(trial: optuna.Trial):
    lr = trial.suggest_float('lr', 1e-4, 5e-4)
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-4)


    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(pcn.parameters(), lr=lr, weight_decay=weight_decay)
    loss = loss_fn(pcn(imgs_tensor.float()), labels_tensor)
    return loss

In [72]:
study = optuna.create_study()
study.optimize(objective, n_trials=100)

[I 2024-07-23 07:19:11,766] A new study created in memory with name: no-name-4dc9fbb9-9928-447b-bee7-74d52762b3a1
[I 2024-07-23 07:19:13,015] Trial 0 finished with value: 1.6096606254577637 and parameters: {'lr': 0.0004276435057690889, 'weight_decay': 4.0454792288467305e-05}. Best is trial 0 with value: 1.6096606254577637.
[I 2024-07-23 07:19:14,674] Trial 1 finished with value: 1.6096606254577637 and parameters: {'lr': 0.00011709515088963664, 'weight_decay': 6.966750848689825e-05}. Best is trial 0 with value: 1.6096606254577637.
[I 2024-07-23 07:19:16,332] Trial 2 finished with value: 1.6096606254577637 and parameters: {'lr': 0.0004850092342922968, 'weight_decay': 7.782781788059048e-05}. Best is trial 0 with value: 1.6096606254577637.
[I 2024-07-23 07:19:17,924] Trial 3 finished with value: 1.6096606254577637 and parameters: {'lr': 0.0002260167360938268, 'weight_decay': 2.237095436209281e-05}. Best is trial 0 with value: 1.6096606254577637.
[I 2024-07-23 07:19:19,608] Trial 4 finished

In [73]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(pcn.parameters(), lr=study.best_params['lr'], weight_decay=study.best_params['weight_decay'])

In [74]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(imgs_tensor, labels_tensor, train_size=0.8, test_size=0.2)

In [75]:
epochs = 100

for _ in range(epochs):
  loss = loss_fn(pcn(X_train.float()), y_train)
  print(f'Epoch {_ + 1} ; Loss {loss}')
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()

Epoch 1 ; Loss 1.6098121404647827
Epoch 2 ; Loss 1.601608395576477
Epoch 3 ; Loss 1.5840002298355103
Epoch 4 ; Loss 1.549574375152588
Epoch 5 ; Loss 1.5050098896026611
Epoch 6 ; Loss 1.4567327499389648
Epoch 7 ; Loss 1.4271376132965088
Epoch 8 ; Loss 1.5158917903900146
Epoch 9 ; Loss 1.3531677722930908
Epoch 10 ; Loss 1.4285609722137451
Epoch 11 ; Loss 1.3127375841140747
Epoch 12 ; Loss 1.3882068395614624
Epoch 13 ; Loss 1.2875455617904663
Epoch 14 ; Loss 1.2610843181610107
Epoch 15 ; Loss 1.254164457321167
Epoch 16 ; Loss 1.2028189897537231
Epoch 17 ; Loss 1.2050408124923706
Epoch 18 ; Loss 1.1586530208587646
Epoch 19 ; Loss 1.148543119430542
Epoch 20 ; Loss 1.113083839416504
Epoch 21 ; Loss 1.115169882774353
Epoch 22 ; Loss 1.073821783065796
Epoch 23 ; Loss 1.0873972177505493
Epoch 24 ; Loss 1.0461342334747314
Epoch 25 ; Loss 1.0556026697158813
Epoch 26 ; Loss 1.0314600467681885
Epoch 27 ; Loss 1.0240470170974731
Epoch 28 ; Loss 1.0220261812210083
Epoch 29 ; Loss 1.005500078201294
Ep

In [76]:
def softmax_accuracy(probs, labels):
    """
    Calculate the accuracy of predictions given softmax probabilities and true labels.

    Parameters:
    probs (torch.Tensor): A tensor of shape (batch_size, num_classes) with softmax probabilities.
    labels (torch.Tensor): A tensor of shape (batch_size,) with true labels.

    Returns:
    float: The accuracy of the predictions.
    """
    # Convert softmax probabilities to predicted class labels
    _, predicted_labels = torch.max(probs, 1)

    # Compare predicted labels to true labels
    correct_predictions = (predicted_labels == labels).sum().item()

    # Calculate accuracy
    accuracy = correct_predictions / labels.size(0)

    return accuracy

In [79]:
softmax_accuracy(pcn(X_train.float()), y_train)

0.94925

In [80]:
softmax_accuracy(pcn(X_test.float()), y_test)

0.939